In [9]:
import sys
import os
import torch
sys.path.append("C:/Users/yuuta/Documents/fashion")
from utils.util import filter_basic_items, open_json
from utils.util import is_target_category
from utils.util import calculate_euclid_sum
from utils.infer import id_to_vector
from utils.util import calc_roc_auc
from utils.util import calc_cw_score

import json
import glob
import random
import pandas as pd
from utils.util import get_category


In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [11]:
filepaths = glob.glob(
    f"C:/Users/yuuta/Documents/fashion/data/test/**/*.json"
)

In [12]:
ver = open_json('C:/Users/yuuta/Documents/fashion/experiments/data/id_to_vector_versatility.json')
com = open_json('C:/Users/yuuta/Documents/fashion/experiments/data/id_to_vector_compatibility.json')

In [13]:
items = {"tops": [], "bottoms": [], "shoes": []}
for fp in filepaths:
    json_dict = open_json(fp)
    if json_dict == None:
        continue
    for item in filter_basic_items(json_dict["items"]):
        category = get_category(item)
        if category == None:
            continue
        items[category].append(str(item['itemId']))

In [14]:
categories = ['tops', 'bottoms', 'shoes']
# 枚数
T = 3

In [15]:
import importlib
import utils
importlib.reload(utils.util)
importlib.reload(utils.versatility)
from utils.util import calc_cw_score
from utils.util import calc_increase_c, get_progress_percent
from utils.versatility import calc_increase_v
from utils.util import scale_increase_versatility
from utils.util import scale_versatility, scale_compatibility
from utils.util import scale_increase_compatibility

In [16]:
inc_ver_scores = []
ver_scores = []
inc_com_scores = []
com_scores = []
dataset = {
    'tops':items["tops"],
    'bottoms':items["bottoms"],
    'shoes':items["shoes"],
}
N = 100
for i in range(N):
    if i % 10 == 0:
        print(get_progress_percent(i, N, div=100), end='')
    
    # cwの初期化
    cw = {
        'tops':random.sample(dataset["tops"], T),
        'bottoms':random.sample(dataset["bottoms"], T),
        'shoes':random.sample(dataset["shoes"], T),
    }
    eps = 10e-3
    increase_score = eps + 1
    pre_cw_score = 0
    while increase_score > eps:
        for cat in categories:
            # そのレイヤを空にする
            cw[cat] = []
            # それぞれのスコアを初期化
            ver_set = set()
            for _ in range(T):
                # 追加するアイテムと、そのアイテムが追加されたことによるCWスコアの上昇値を初期化
                add_item = None
                max_increase_score = -1
                for item in dataset[cat]:
                    if item in cw[cat]:
                        continue
                    # 増加分を計算
                    increase_c, increase_v = 0, 0
                    if com != None:
                        increase_c = calc_increase_c(cw, com, item, cat)
                    if ver != None:
                        cover_clusters = calc_increase_v(torch.tensor(ver[item]))
                        increase_v = len(cover_clusters.union(ver_set)) - len(ver_set)

                    inc_com_scores.append(increase_c)
                    inc_ver_scores.append(increase_v)
                    increase_c, increase_v = scale_increase_compatibility(increase_c), scale_increase_versatility(increase_v)
                    # increase_v = 0
                    # increase_c = 0
                    increase = increase_c + increase_v
                    if increase > max_increase_score:
                        max_increase_score = increase
                        add_item = item

                if com != None:
                    increase_c = calc_increase_c(cw, com, add_item, cat)
                    increase_c = scale_increase_compatibility(increase_c)
                    # inc_com_scores.append(increase_c)
                    
                if ver != None:
                    cover_clusters = calc_increase_v(torch.tensor(ver[add_item]))
                    # increase_v =  len(ver_set.union(
                    #     cover_clusters
                    # )) - len(ver_set)
                    # increase_v = scale_increase_versatility(increase_v)

                    ver_set = ver_set.union(
                        cover_clusters
                    )

                cw[cat].append(add_item)

        c, v = calc_cw_score(cw, com, ver)
        ver_scores.append(v)
        com_scores.append(c)
        c, v = scale_compatibility(c), scale_versatility(v)
        score = c + v
        increase_score = score - pre_cw_score
        pre_cw_score = score
        # print(c, v)

【                                                                                                    】[('9071925', '11695462', '37510596'), ('9071925', '41632926', '37510596'), ('9071925', '28570274', '37510596'), ('39560414', '11695462', '37510596'), ('39560414', '41632926', '37510596'), ('39560414', '28570274', '37510596'), ('13973444', '11695462', '37510596'), ('13973444', '41632926', '37510596'), ('13973444', '28570274', '37510596')]


In [ ]:

# df1 = pd.DataFrame({'compatibiltity': list(map(scale_compatibility, com_scores)), 'versatility': list(map(scale_versatility,ver_scores))})
# df2 = pd.DataFrame({'compatibiltity':  list(map(scale_increase_compatibility, inc_com_scores)), 'versatility': list(map(scale_increase_versatility,inc_ver_scores))})
df1 = pd.DataFrame({'compatibiltity': com_scores, 'versatility': ver_scores})
df2 = pd.DataFrame({'compatibiltity':  inc_com_scores, 'versatility': inc_ver_scores})
# df1 = pd.Series(ver_scores)
# df2 = pd.Series([c.item() for c in inc_ver_scores])
# df2 = pd.Series(inc_ver_scores)
# df2 = pd.DataFrame({'inc':  list(map(scale_increase_compatibility, inc_com_scores)), 'cw': list(map(scale_compatibility, com_scores))})
print(df1.describe())
print(df2.describe())

       compatibiltity  versatility
count       10.000000    10.000000
mean         3.987917     1.287487
std          1.262666     1.144187
min          2.054351     0.000000
25%          3.086953     0.123454
50%          3.984471     1.359611
75%          4.685213     2.130378
max          5.923189     3.070082
       compatibiltity  versatility
count      234.000000   234.000000
mean         4.805071     4.868959
std          1.160991     1.100510
min          2.997178     3.315154
25%          3.902596     3.872068
50%          4.655278     4.634721
75%          5.705053     6.294822
max          7.681937     6.294822


In [ ]:
com_scores[:3]

[74208.45125706775, 74893.0940488335, 38796.05644528226]

In [ ]:
print(inc_ver_scores[:5])

[6.294821943820937, 4.890121185095234, 3.896898426400292, 6.294821943820937, 4.677287736803461]


In [ ]:
com = df.describe()['compatibiltity']
ver = df.describe()['versatility']

NameError: name 'df' is not defined

In [ ]:
print(
    f'min_value = {com["min"]}\nmax_value = {com["max"]}\n'
)

In [ ]:
print(
    f'min_value = {ver["min"]}\nmax_value = {ver["max"]}\n'
)